In [ ]:
!git config --global --unset http.proxy 
!git config --global --unset https.proxy

In [ ]:
!pip install q tensorflow==2.1
!pip install q keras==2.3.1

In [ ]:
# !pip install -U albumentations
!pip install -U segmentation-models

In [ ]:
import keras
print(keras.__version__)
import tensorflow as tf
print(tf.__version__)

In [ ]:
import albumentations as A
import segmentation_models as sm

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATA_DIR = '/content/oral-cancer-data/'

# load repo with data if it is not exists
if not os.path.exists(DATA_DIR): print('load data!')

In [ ]:
print("Number of images")
!ls ./oral-cancer-data/train/images/ | wc -l
print("Number of masks")
!ls ./oral-cancer-data/train/labels/ | wc -l

In [ ]:
x_train_dir = os.path.join(DATA_DIR, 'train', 'images')
y_train_dir = os.path.join(DATA_DIR, 'train', 'labels')

x_test_dir = os.path.join(DATA_DIR, 'test', 'images')
y_test_dir = os.path.join(DATA_DIR, 'test', 'labels')

In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

In [ ]:
# classes for data loading and preprocessing
class Dataset:
    """Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['lesion']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))

        mask = cv2.imread(self.masks_fps[i], 0)
        mask = cv2.resize(mask, (256, 256))
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        mask = np.where(mask>0, 0, 1)
        
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return batch
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [ ]:
# Lets look at data we have
dataset = Dataset(x_train_dir, y_train_dir, classes=['lesion'])

image, mask = dataset[7] # get some sample
print("image shape: {} and mask shape: {}".format(image.shape,mask.shape))
visualize(
    image=image, 
    lesion_mask=mask[..., 0].squeeze()
)

In [ ]:
# def round_clip_0_1(x, **kwargs):
#     return x.round().clip(0, 1)

# # define heavy augmentations
# def get_training_augmentation():
#     train_transform = [

#         A.HorizontalFlip(p=0.5),

#         A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.25, rotate_limit=0, p=1, border_mode=0),

#         A.PadIfNeeded(min_height=256, min_width=256, always_apply=True, border_mode=0),
#         # A.RandomCrop(height=256, width=256, always_apply=True),

#         # A.IAAAdditiveGaussianNoise(p=0.2),
#         # A.IAAPerspective(p=0.5),

#         # A.OneOf(
#         #     [
#         #         A.CLAHE(p=1),
#         #         A.RandomBrightness(p=1),
#         #         A.RandomGamma(p=1),
#         #     ],
#         #     p=0.9,
#         # ),

#         # A.OneOf(
#         #     [
#         #         A.IAASharpen(p=1),
#         #         A.Blur(blur_limit=3, p=1),
#         #         A.MotionBlur(blur_limit=3, p=1),
#         #     ],
#         #     p=0.9,
#         # ),

#         # A.OneOf(
#         #     [
#         #         A.RandomContrast(p=1),
#         #         A.HueSaturationValue(p=1),
#         #     ],
#         #     p=0.9,
#         # ),
#         A.Lambda(mask=round_clip_0_1)
#     ]
#     return A.Compose(train_transform)


# def get_validation_augmentation():
#     """Add paddings to make image shape divisible by 32"""
#     test_transform = [
#         A.PadIfNeeded(256, 256)
#     ]
#     return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [ ]:
# # Lets look at augmented data we have
# dataset = Dataset(x_train_dir, y_train_dir, classes=['lesion'), augmentation=get_training_augmentation())

# image, mask = dataset[7] # get some sample
# print("image shape: {} and mask shape: {}".format(image.shape,mask.shape))
# visualize(
#     image=image, 
#     lesion_mask=mask[..., 0].squeeze()
# )

In [ ]:
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 8
CLASSES = ['lesion']
LR = 0.0001
EPOCHS = 400

preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

In [ ]:
optim = keras.optimizers.Adam(LR)

total_loss = sm.losses.binary_focal_dice_loss + (1 * sm.losses.binary_crossentropy)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [ ]:
# Dataset for train images
train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    classes=CLASSES, 
    # augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocess_input)
)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
print(train_dataloader[0][0].shape, train_dataloader[0][1].shape)
# check shapes for errors
assert train_dataloader[0][0].shape == (BATCH_SIZE, 256, 256, 3)
assert train_dataloader[0][1].shape == (BATCH_SIZE, 256, 256, n_classes)

In [ ]:
# def schedule(EPOCHS, LR):
#   if EPOCHS < 150:
#     return LR;
#   elif LR < 1e-6:
#     return 1e-6
#   else:
#     return LR * 0.3

# # define callbacks for learning rate scheduling
# callbacks = [
#     keras.callbacks.LearningRateScheduler(schedule, verbose=1)
# ]

In [ ]:
# train model
history = model.fit_generator(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs=EPOCHS
    # callbacks=callbacks,
)

In [ ]:
model.save_weights("unet_{}_weights.h5".format(BACKBONE))

In [ ]:
model.summary()

In [ ]:
# Plot training iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['f1-score'])
plt.title('Model scores')
plt.ylabel('Scores')
plt.xlabel('Epoch')
plt.legend(['IOU score', 'F1 score'], loc='upper left')

# Plot training loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [ ]:
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=CLASSES, 
    # augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input)
)

test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# load best weights
model.load_weights('/content/unet_efficientnetb3_weights.h5')

In [ ]:
scores_100 = scores
print(scores_100)

In [ ]:
scores = model.evaluate_generator(test_dataloader)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__, value))

In [ ]:
n = 5
ids = np.random.choice(np.arange(len(test_dataset)), size=n, replace=False)

for i in ids:
    
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )

In [ ]:
n = 22 # on train dataset
ids = np.random.choice(np.arange(len(train_dataset)), size=n, replace=False)

for i in ids:
    
    image, gt_mask = train_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )